# Clarkston Consulting Interview Assignment
This notebook analyzes the CustomerExtract.csv file and provides the following:
<ol>
<li>Reads in the CustomerExtract.csv</li>
<li>Completes an initial profile of the data and performs data quality checks that you think are needed e.g. Are there any fields in the extract that are populated with data that are not flagged in the spec (Field Utilized in LEGACY System)<ol><li>Quality checks should be visible in the notebook and output for traceability</li></ol></li>
<li>Cleanup any data quality issues, such as special characters in the Name 1 field</li>
<li>Perform the transformations included in the spec</li>
<li>Output a flat file named CustomerLoad.csv</li>
</ol>

## Imports

In [1]:
import pandas as pd
import regex as re

/var/folders/f7/p92k225d4t57x80m5zl_8jjh0000gn/T/ipykernel_63089/4113098713.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Prep Spec Document Into Dataframe For Dataset Analysis

### Read Spec Doc Into DataFrame

In [2]:
# Specify Usable Column Names For Later
customer_spec_columns = [
    'field_label',
    'utilized_in_legacy_system',
    'utilized_in_new_system',
    'table_name',
    'field_name',
    'field_length',
    'field_datatype',
    'field_used',
    'field_description',
    'required_in_legacy_system',
    'required_in_new_system',
    'fit_or_gap',
    'etl_logic'
]

# Read Excel Doc into DF
customer_spec_df = pd.read_excel(
    'Dataset/CustomerSpec.xlsx',
    header=None, # Replace Header With Spec Columns Dictionary
    skipfooter=0,
    skiprows=1,
    sheet_name='CustomerSpec',
    names=customer_spec_columns,
    true_values=['Y'],
    false_values=['N']
)

/Users/tvisser/Library/Mobile Documents/com~apple~CloudDocs/Projects/clarkston_consulting/env/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


### Add "Pythonic" Label For Each Field

In [3]:
# Add a lookup field to map to the extract
customer_spec_df['extract_field_name'] = customer_spec_df['field_label'].str.lower().replace({
    r'[^A-Z|a-z|0-9]$': '',
    r'[^A-Z|a-z|0-9]': '_'
    }, regex=True
)

### Filter Spec To Just Used Columns

In [4]:
customer_spec_required_columns_df = customer_spec_df.loc[customer_spec_df['field_used'] == True]

## Extract Analysis

In [5]:
# Read CSV into Pandas dataframe
customer_extract_df = pd.read_csv(
    'Dataset/CustomerExtract.csv',
    sep=',',
    header=0,
    true_values=['Y'],
    false_values=['N'],
    dtype=object
)

### Add Static Required Fields Specified In Selection Logic

In [6]:
# company_code
customer_extract_df['BUKRS'] = 'G100'

# sales_organization
customer_extract_df['VKORG'] = 'G100'

# distrubtion_channel
customer_extract_df['VTWEG'] = '20'

# division
customer_extract_df['SPART'] = '10'

### Find Potentially Used Fields From Legacy System Not Required In Spec

In [7]:
unused_fields = customer_spec_df.loc[(customer_spec_df['field_used'] == False) & (customer_spec_df['utilized_in_legacy_system'] == True)]

if unused_fields.empty:
    print('ALL FIELDS USED IN LEGACY SYSTEM ARE IN USE IN THE NEW SYSTEM')
else:
    for field in unused_fields['field_name'].itertuples():
        rows_with_data = len(customer_spec_df.loc[customer_spec_df[field].dropna()])
        if rows_with_data > 0:
            print(field, 'HAS DATA UNUSED IN NEW SYSTEM.')

ALL FIELDS USED IN LEGACY SYSTEM ARE IN USE IN THE NEW SYSTEM


### Remove Unused Columns For Simplicity

In [8]:
cols_to_drop = [col for col in customer_extract_df if col not in customer_spec_required_columns_df['field_name'].to_list()]

customer_extract_df.drop(columns=cols_to_drop, inplace=True)

print("DROPPING", len(cols_to_drop), "UNUSED COLUMNS")

DROPPING 137 UNUSED COLUMNS


### Rename Columns To Friendly Names From Spec

In [9]:
# Rename Extract Columns From Spec
customer_extract_column_dict = {}

for index, field_name, field_label in customer_spec_required_columns_df[['field_name', 'field_label']].itertuples():
    # Remove trailing special characters
    field_label = re.sub(r'[^A-Z|a-z|0-9]$', '', field_label)
    # Replace remaining special characters with underscores
    field_label = re.sub(r'[^A-Z|a-z|0-9]', '_', field_label)
    # Lower Case field label
    field_label = field_label.lower()
    if field_label not in customer_extract_column_dict.values() and not customer_extract_column_dict.get(field_name):
        customer_extract_column_dict.update({field_name: field_label})

customer_extract_df.rename(columns=customer_extract_column_dict, inplace=True)

### Cast Fields To Proper Types Based On Specs

In [10]:
# Update Data Types Using Spec
data_type_map = {
    'CHAR': "str",
    'NUMC': "numeric",
    'DATS': "datetime",
    'LANG': "str",
    'CURR': "numeric",
    'CUKY': "numeric",
    'TIMS': "datetime",
    'QUAN': "numeric",
    'UNIT': "numeric",
}

for index, field_label, field_datatype in customer_spec_required_columns_df[['extract_field_name', 'field_datatype']].itertuples():
    if field_label in customer_extract_df.columns:
        data_type = data_type_map[field_datatype]
        if data_type == "datetime":
            customer_extract_df[field_label] = pd.to_datetime(customer_extract_df[field_label], errors='coerce')
        elif data_type == "numeric":
            customer_extract_df[field_label] = pd.to_numeric(customer_extract_df[field_label], errors='coerce')
        elif data_type == "str":
            customer_extract_df[field_label] = customer_extract_df[field_label].astype(str)

# Replace Nulls With Proper Nulls
customer_extract_df.replace("nan", None, inplace=True)

/var/folders/f7/p92k225d4t57x80m5zl_8jjh0000gn/T/ipykernel_63089/3255789744.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  customer_extract_df[field_label] = pd.to_datetime(customer_extract_df[field_label], errors='coerce')


### Check For Missing Required Fields In Extract

In [11]:
# Find any required columns in the extract missing from the spec
required_fields = customer_spec_df['extract_field_name'].loc[customer_spec_df['required_in_new_system'] == True]

missing_fields = []
for col in required_fields:
    if col not in customer_extract_df.columns:
        missing_fields.append(col)

if missing_fields:
    print("THE FOLLOWING COLUMNS ARE MISSING FROM THE EXTRACT AND ARE REQUIRED IN THE LEGACY SYSTEM")
    [print(fields) for fields in missing_fields]
else:
    print('ALL REQUIRED FIELDS IN NEW SYSTEM EXIST IN EXTRACT')

ALL REQUIRED FIELDS IN NEW SYSTEM EXIST IN EXTRACT


### Check For Truncation In String Columns

In [12]:
# Check For Truncation
field_data_types = customer_spec_df[['extract_field_name', 'field_datatype', 'field_length']].loc[customer_spec_df['utilized_in_new_system'] == True]

has_truncation = False
for index, field_name, field_datatype, field_length in field_data_types.itertuples():
    # Check for truncation
    if data_type_map[field_datatype] == "str" and field_name in customer_extract_df.columns:
        truncated_records = customer_extract_df[field_name].loc[
            (customer_extract_df[field_name].astype(str).str.len() > field_length) & (customer_extract_df[field_name].isna() == False)
        ].drop_duplicates()

        if not truncated_records.empty:
            print(field_name, 'has', len(truncated_records), 'truncated records. Max length for column is', field_length)
            has_trunction = True

if not has_truncation:
    print('NO TRUNCATED COLUMNS FOUND')


NO TRUNCATED COLUMNS FOUND


### Remove Special Characters From String Fields

In [13]:
string_columns = [col for col in customer_extract_df.select_dtypes(include='object').columns]

customer_extract_df[string_columns] = customer_extract_df[string_columns].replace(r'[^A-Z|a-z|0-9|\,]', '', regex=True)

## Output File

### Create Output With Original Column Names

In [14]:
original_columns_dict = {}

for original_col, friendly_col in customer_extract_column_dict.items():
    original_columns_dict[friendly_col] = original_col

customer_extract_df.rename(columns=original_columns_dict).to_csv('CustomerLoad.csv', index=False)

### Create Output With Friendly Column Names

In [15]:
customer_extract_df.to_csv('CustomerLoadFriendlyNames.csv', index=False)

# My Process

<ul>
<li>Convert the spec into a usable format</li>
<li>Rename the extract fields with friendly labels from spec</li>
<li>Check for any issues in the extract such as truncation, bad types, missing fields, etc</li>
<li>Remove special characters from the string fields.</li>
</ul>

# Found Problems (Fixed By Filtering To Used Fields)

<ul>
    <li>Account Group and Customer Account have ambiguous field names in the extract. There are two fields in the spec with distinct table/column name combos but the extract doesn't specify which is which.</li>
    <li>Duplicate SAP Field Descriptions</li>
    <li>Leading and trailing special characters for the field descriptions. This is only an issue due to converting the field names in the extract to friendly terms</li>
</ul>